In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as sns
%matplotlib inline
import nltk
from nltk import sent_tokenize,word_tokenize,pos_tag,pos_tag_sents
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords,wordnet
import string

In [2]:
df2=pd.read_csv('fake.csv')
df2.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [3]:
df=df2[['title','text','spam_score']]
df.head()

,title,text,spam_score
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0.000
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0.000
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,0.000
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,0.068
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0.865


In [4]:
df.shape

(12999, 3)

In [5]:
df.isnull().sum()

title         680
text           46
spam_score      0
dtype: int64

In [6]:
df.dropna(axis=0,inplace=True)
df.shape

/home/abhishekverma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(12273, 3)

In [7]:
df['real']=df['spam_score']>0.5
df.head()

/home/abhishekverma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,title,text,spam_score,real
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0.000,False
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0.000,False
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,0.000,False
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,0.068,False
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0.865,True


In [8]:
def strtobool(str):
    if(str==False):
        return 0
    else:
        return 1
df['real']=df['real'].apply(strtobool)
df.head()

/home/abhishekverma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,title,text,spam_score,real
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0.000,0
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0.000,0
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,0.000,0
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,0.068,0
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,0.865,1


In [9]:
del df['spam_score']
df.head()

,title,text,real
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,0
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,0
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,0
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,0
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,1


In [10]:
df['txt']=df['title']+df['text']
del df['title']
del df['text']
df.head()

/home/abhishekverma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,real,txt
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...
1,0,Re: Why Did Attorney General Loretta Lynch Ple...
2,0,BREAKING: Weiner Cooperating With FBI On Hilla...
3,0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...
4,1,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...


In [17]:
stri=df.iloc[0]
word_tokenize(stri[1])

['Muslims',
 'BUSTED',
 ':',
 'They',
 'Stole',
 'Millions',
 'In',
 'Gov',
 '’',
 't',
 'BenefitsPrint',
 'They',
 'should',
 'pay',
 'all',
 'the',
 'back',
 'all',
 'the',
 'money',
 'plus',
 'interest',
 '.',
 'The',
 'entire',
 'family',
 'and',
 'everyone',
 'who',
 'came',
 'in',
 'with',
 'them',
 'need',
 'to',
 'be',
 'deported',
 'asap',
 '.',
 'Why',
 'did',
 'it',
 'take',
 'two',
 'years',
 'to',
 'bust',
 'them',
 '?',
 'Here',
 'we',
 'go',
 'again',
 '…another',
 'group',
 'stealing',
 'from',
 'the',
 'government',
 'and',
 'taxpayers',
 '!',
 'A',
 'group',
 'of',
 'Somalis',
 'stole',
 'over',
 'four',
 'million',
 'in',
 'government',
 'benefits',
 'over',
 'just',
 '10',
 'months',
 '!',
 'We',
 '’',
 've',
 'reported',
 'on',
 'numerous',
 'cases',
 'like',
 'this',
 'one',
 'where',
 'the',
 'Muslim',
 'refugees/immigrants',
 'commit',
 'fraud',
 'by',
 'scamming',
 'our',
 'system…It',
 '’',
 's',
 'way',
 'out',
 'of',
 'control',
 '!',
 'More',
 'Related']

In [19]:
documents=[]
for i in range(df.shape[0]):
    line=df.iloc[i]
    txt=word_tokenize(line[1])
    r=line[0]
    tup=(txt,r)
    documents.append(tup)
    
documents[0:5]

[(['Muslims',
   'BUSTED',
   ':',
   'They',
   'Stole',
   'Millions',
   'In',
   'Gov',
   '’',
   't',
   'BenefitsPrint',
   'They',
   'should',
   'pay',
   'all',
   'the',
   'back',
   'all',
   'the',
   'money',
   'plus',
   'interest',
   '.',
   'The',
   'entire',
   'family',
   'and',
   'everyone',
   'who',
   'came',
   'in',
   'with',
   'them',
   'need',
   'to',
   'be',
   'deported',
   'asap',
   '.',
   'Why',
   'did',
   'it',
   'take',
   'two',
   'years',
   'to',
   'bust',
   'them',
   '?',
   'Here',
   'we',
   'go',
   'again',
   '…another',
   'group',
   'stealing',
   'from',
   'the',
   'government',
   'and',
   'taxpayers',
   '!',
   'A',
   'group',
   'of',
   'Somalis',
   'stole',
   'over',
   'four',
   'million',
   'in',
   'government',
   'benefits',
   'over',
   'just',
   '10',
   'months',
   '!',
   'We',
   '’',
   've',
   'reported',
   'on',
   'numerous',
   'cases',
   'like',
   'this',
   'one',
   'where',
   '

In [35]:
stops=stopwords.words('english')
punctuations=list(string.punctuation)
stops.extend(punctuations)
print(len(stops))
stops

211


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [36]:
lemmatizer=WordNetLemmatizer()
lemmatizer.lemmatize(documents[0][0][5])

'Millions'

In [41]:
pos_tag(['millions'])[0][1]

'NNS'

In [43]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [44]:
dir(wordnet)

['ADJ',
 'ADJ_SAT',
 'ADV',
 'MORPHOLOGICAL_SUBSTITUTIONS',
 'NOUN',
 'VERB',
 '_ENCODING',
 '_FILEMAP',
 '_FILES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_compute_max_depth',
 '_data_file',
 '_data_file_map',
 '_encoding',
 '_exception_map',
 '_fileids',
 '_get_root',
 '_key_count_file',
 '_key_synset_file',
 '_lang_data',
 '_lemma_pos_offset_map',
 '_lexnames',
 '_load_exception_map',
 '_load_lang_data',
 '_load_lemma_pos_offset_map',
 '_max_depth',
 '_morphy',
 '_omw_reader',
 '_pos_names',
 '_pos_numbers',
 '_root',
 '_synset_from_pos_and_line',
 '_synset_from_pos_and_offset',
 '_synset_offset_cache',
 '_tagset',
 '_unload',
 'abspath',
 '

In [45]:
def simple_pos(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        

In [50]:
simple_pos('JJ')

'a'

In [47]:
lemmatizer.lemmatize('better',pos=simple_pos(pos_tag(['better'])[0][1]))

'well'

In [51]:
def clean_review(words):
    op=[]
    for w in words:
        if w not in stops:
            p=pos_tag([w])
            cw=lemmatizer.lemmatize(w,pos=simple_pos(p[0][1]))
            op.append(cw)
    return op


In [52]:
documents=[(clean_review(doc),tar) for doc,tar in documents]
documents[0:5]

[(['Muslims',
   'BUSTED',
   'They',
   'Stole',
   'Millions',
   'In',
   'Gov',
   '’',
   'BenefitsPrint',
   'They',
   'pay',
   'back',
   'money',
   'plus',
   'interest',
   'The',
   'entire',
   'family',
   'everyone',
   'come',
   'need',
   'deport',
   'asap',
   'Why',
   'take',
   'two',
   'year',
   'bust',
   'Here',
   'go',
   '…another',
   'group',
   'steal',
   'government',
   'taxpayer',
   'A',
   'group',
   'Somalis',
   'stole',
   'four',
   'million',
   'government',
   'benefit',
   '10',
   'month',
   'We',
   '’',
   'report',
   'numerous',
   'case',
   'like',
   'one',
   'Muslim',
   'refugees/immigrants',
   'commit',
   'fraud',
   'scamming',
   'system…It',
   '’',
   'way',
   'control',
   'More',
   'Related'],
  0),
 (['Re',
   'Why',
   'Did',
   'Attorney',
   'General',
   'Loretta',
   'Lynch',
   'Plead',
   'The',
   'Fifth',
   'Why',
   'Did',
   'Attorney',
   'General',
   'Loretta',
   'Lynch',
   'Plead',
   'The',
   